In [ ]:
!pip install roboflow torch torchvision opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 115.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
# Library to import dataset
from roboflow import Roboflow
rf = Roboflow(api_key="qb83OLxB1WoQkLd30fIQ")
project = rf.workspace("vitto-rossetto-nbvy6").project("dog-pose-ntugs")
version = project.version(1)
dataset = version.download("folder")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dog-pose-1 in folder:: 100%|██████████| 1539/1539 [00:00<00:00, 4542.18it/s]


In [ ]:
# Import for training purposes
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_dir = "dog-pose-1"

# Define transformations for training and validation sets
train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

# Validation transformations (no augmentation)
val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

# Load datasets
train_ds = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_tf)
val_ds   = datasets.ImageFolder(os.path.join(data_dir, "valid"), transform=val_tf)

# Create data loaders
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=32, shuffle=False)

# Print class names
print(train_ds.classes)


['lie', 'sit', 'stand']


In [ ]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.4 MB/s eta 0:00:00


In [ ]:
# Import YOLO from ultralytics
from ultralytics import YOLO

# load pretrained tiny classifier
model = YOLO("yolov8n-cls.pt")

# dataset folder (adjust if name differs)
data_path = "dog-pose-1"

# train the model
model.train(
    data=data_path,
    epochs=100,
    imgsz=224,
    batch=32
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dog-pose-1, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0,

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7977f0ea6780>
curves: []
curves_results: []
fitness: 0.991830050945282
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.983660101890564, 'metrics/accuracy_top5': 1.0, 'fitness': 0.991830050945282}
save_dir: PosixPath('/content/runs/classify/train')
speed: {'preprocess': 0.0011433137254190275, 'inference': 12.136358588232646, 'loss': 2.408169582410064e-05, 'postprocess': 5.3290846361740633e-05}
task: 'classify'
top1: 0.983660101890564
top5: 1.0

In [ ]:
# validate the model
model.val()

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLOv8n-cls summary (fused): 30 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs
train: /content/dog-pose-1/train... found 1223 images in 3 classes ✅ 
val: /content/dog-pose-1/valid... found 306 images in 3 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 195.5±73.4 MB/s, size: 6.5 KB)
val: Scanning /content/dog-pose-1/valid... 306 images, 0 corrupt: 100% ━━━━━━━━━━━━ 306/306 310.1Kit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 20/20 3.1it/s 6.5s
                   all      0.984          1
Speed: 0.0ms preprocess, 13.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/classify/val


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7977e08a2d20>
curves: []
curves_results: []
fitness: 0.991830050945282
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.983660101890564, 'metrics/accuracy_top5': 1.0, 'fitness': 0.991830050945282}
save_dir: PosixPath('/content/runs/classify/val')
speed: {'preprocess': 0.0024015947679336203, 'inference': 13.081502568626938, 'loss': 8.319281786337824e-05, 'postprocess': 0.00027973529463481717}
task: 'classify'
top1: 0.983660101890564
top5: 1.0

In [ ]:
# export the model to ONNX and TFLite formats
model.export(format="onnx")
model.export(format="tflite")

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/content/runs/classify/train/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 3) (2.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 3.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.73...
ONNX: export success ✅ 4.9s, saved as '/content/runs/classify/train/weights/best.onnx' (5.5 MB)

Export complete (5.0s)
Results saved to /content/runs/classify/train/weights
Predict:         yolo predict task=classify model=/content/runs/classify/train/weights/best.onnx imgsz=224 

'/content/runs/classify/train/weights/best_saved_model/best_float32.tflite'

In [ ]:
# load the best model and make a prediction
model = YOLO("runs/classify/train/weights/best.pt")

result = model("sampleDogLie.jpeg")
print(result[0].probs)


image 1/1 /content/sampleDogLie.jpeg: 224x224 lie 1.00, sit 0.00, stand 0.00, 16.6ms
Speed: 3.7ms preprocess, 16.6ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
ultralytics.engine.results.Probs object with attributes:

data: tensor([9.9911e-01, 8.6989e-04, 1.5928e-05])
orig_shape: None
shape: torch.Size([3])
top1: 0
top1conf: tensor(0.9991)
top5: [0, 1, 2]
top5conf: tensor([9.9911e-01, 8.6989e-04, 1.5928e-05])
